# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [112]:
# import libraries
import re
import pickle
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', 100)

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sqlalchemy import create_engine

from sklearn.metrics import (confusion_matrix, classification_report,
                             precision_recall_fscore_support, make_scorer,
                             precision_score, recall_score, roc_auc_score,
                             f1_score, accuracy_score)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import (CountVectorizer, TfidfTransformer,
                                             HashingVectorizer)
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier

import seaborn as sns
cm = sns.light_palette("green", as_cmap=True)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gabrielestivalet/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gabrielestivalet/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/gabrielestivalet/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gabrielestivalet/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
%ls data/

In [2]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql('select * from main_table', con=engine)
df = df.drop('original', axis=1).dropna()
print(df.shape)

X = df['message']
Y = df.drop(['id', 'message', 'genre'], axis=1)

(26207, 39)


### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word.lower().strip()) for word in tokens if word not in stop_words]

    return tokens

#X['tokenized'] = X['message'].apply(tokenize)
#X.head(10)

In [5]:
#X.drop('tokenized', axis=1, inplace=True)
#X.shape

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [66]:
multi_clf = MultiOutputClassifier(RandomForestClassifier(random_state=42))

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', multi_clf)
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [68]:
y_pred = pipeline.predict(X_test)

# Get results and add them to a dataframe.
def generate_classification_report(y_test, y_pred):
    classification_report_df = pd.DataFrame()
    
    # loop through categories and add results
    i=0
    for c in y_test.columns:
        precision, recall, f1_score, support = (precision_recall_fscore_support(
            y_test[c], y_pred[:,i], average='weighted'))
        
        # set values in dataframe
        classification_report_df.at[i, 'category' ] = c
        classification_report_df.at[i, 'f1_score'  ] = f1_score
        classification_report_df.at[i, 'precision'] = precision
        classification_report_df.at[i, 'recall'   ] = recall
        i+=1
    
    # print mean results
    print('Mean f_score:', classification_report_df['f1_score'].mean())
    print('Mean precision:', classification_report_df['precision'].mean())
    print('Mean recall:', classification_report_df['recall'].mean())
    
    # organize df for display
    classification_report_df.sort_values(by='f1_score', ascending=False, inplace=True)
    classification_report_df.set_index('category', inplace=True)
    
    return classification_report_df

classification_report_df = generate_classification_report(y_test, y_pred)
classification_report_df.T.style.background_gradient(cmap=cm, axis=1)

Mean f_score: 0.900253206213286
Mean precision: 0.8880983664587077
Mean recall: 0.9224108694730595


/Users/gabrielestivalet/anaconda/envs/snakes/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


category,child_alone,offer,shops,tools,fire,hospitals,aid_centers,missing_people,clothing,cold,electricity,security,money,search_and_rescue,military,refugees,other_infrastructure,death,transport,medical_products,other_weather,buildings,water,infrastructure_related,earthquake,medical_help,floods,storm,shelter,food,other_aid,request,direct_report,weather_related,related,aid_related
f1_score,1,0.993836,0.992693,0.991995,0.985462,0.98299,0.981533,0.98109,0.976657,0.970273,0.969581,0.969317,0.965003,0.959658,0.950358,0.941994,0.933394,0.930832,0.928647,0.927044,0.923954,0.91986,0.909105,0.901022,0.885948,0.881767,0.874954,0.865494,0.865342,0.843781,0.811002,0.773049,0.742269,0.673985,0.671484,0.533743
precision,1,0.991877,0.990357,0.989346,0.980761,0.977489,0.975726,0.974976,0.976715,0.961196,0.959959,0.9625,0.95786,0.949434,0.935174,0.927458,0.915907,0.911118,0.910339,0.908087,0.902149,0.900461,0.887784,0.878433,0.890959,0.8528,0.844113,0.864802,0.838133,0.816159,0.779361,0.760105,0.728278,0.683269,0.652515,0.535941
recall,1,0.995803,0.99504,0.994659,0.990207,0.988554,0.987409,0.987282,0.98423,0.979524,0.979397,0.978888,0.975582,0.971894,0.966043,0.960066,0.95358,0.951418,0.950782,0.947857,0.94684,0.941498,0.935521,0.92878,0.911484,0.914918,0.911484,0.903599,0.902709,0.888465,0.858451,0.822841,0.797914,0.726059,0.714486,0.563525


### 6. Improve your model
Use grid search to find better parameters. 

In [64]:
parameters = {'tfidf__norm':['l1','l2'],
              
              # get 'estimator' from MultiOutputClassifier
              'clf__estimator__n_estimators':[100],
              'clf__estimator__max_depth':[None, 4, 6, 8, 10]}

grid = GridSearchCV(pipeline, param_grid=parameters, cv=3, n_jobs=-1, verbose=1)
grid.fit(X_train, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  7.6min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'tfidf__norm': ['l1', 'l2'], 'clf__estimator__n_estimators': [100], 'clf__estimator__max_depth': range(3, 16, 4)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [65]:
y_pred = grid.predict(X_test)
classification_report_df = generate_classification_report(y_test, y_pred)
classification_report_df.T.style.background_gradient(cmap=cm, axis=1)

/Users/gabrielestivalet/anaconda/envs/snakes/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Mean f_score: 0.8944036563009993
Mean precision: 0.8745642060607111
Mean recall: 0.9271093871437253


category,child_alone,offer,shops,tools,fire,hospitals,aid_centers,missing_people,clothing,cold,electricity,security,money,search_and_rescue,military,refugees,other_infrastructure,death,transport,medical_products,other_weather,buildings,water,infrastructure_related,medical_help,floods,shelter,earthquake,storm,food,other_aid,request,direct_report,related,weather_related,aid_related
f1_score,1,0.9939,0.992757,0.991995,0.985525,0.983054,0.981724,0.981154,0.976408,0.970719,0.969771,0.969392,0.965037,0.96012,0.950867,0.942388,0.934302,0.932424,0.929233,0.929045,0.925482,0.922671,0.910513,0.90305,0.885012,0.877228,0.867244,0.860603,0.858761,0.844421,0.813915,0.758022,0.726735,0.652082,0.611912,0.441068
precision,1,0.991877,0.990358,0.989346,0.980762,0.97749,0.975731,0.974977,0.968709,0.961213,0.959966,0.959468,0.953746,0.947299,0.935207,0.924171,0.913686,0.911256,0.907133,0.906891,0.902295,0.898674,0.883069,0.873534,0.850627,0.840802,0.828255,0.819943,0.817641,0.7998,0.762262,0.694974,0.658156,0.5728,0.80096,0.351233
recall,1,0.99593,0.995167,0.994659,0.990334,0.988681,0.987791,0.987409,0.98423,0.980415,0.979779,0.979524,0.976599,0.973293,0.967061,0.961338,0.955869,0.954597,0.952435,0.952308,0.949892,0.947984,0.939718,0.934631,0.922294,0.916953,0.910085,0.905507,0.904235,0.894315,0.873076,0.833651,0.811268,0.756836,0.725423,0.592649


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [113]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    #"""
    #Feature extraction class based on Udacity's coursework template.
    #"""
    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [123]:
list(range(2,13,2))

[2, 4, 6, 8, 10, 12]

In [139]:
import warnings
warnings.filterwarnings("ignore")

multi_clf = MultiOutputClassifier(XGBClassifier(objective='binary:logistic',
                                                random_state=42))

pipe = Pipeline([ # Feature Union
        ('features', FeatureUnion([
            
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            #('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', multi_clf)
    ])



# parameters
params = { # XGBoost Classifier 
          'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
          #'features__text_pipeline__vect__max_df': (0.75, 1.0),
          #'features__text_pipeline__vect__min_df': (0.75, 1.0),
          'features__text_pipeline__vect__max_features': (None, 5000),
          'features__text_pipeline__tfidf__use_idf': (True, False),
          #'features__text_pipeline__tfidf__norm':['l1','l2'],
    
          #'features__transformer_weights': (
          #      {'text_pipeline': 1, 'starting_verb': 0.5},
          #      {'text_pipeline': 0.5, 'starting_verb': 1},
          #      {'text_pipeline': 0.8, 'starting_verb': 1},
          #    ),
          
          'clf__estimator__n_estimators': [5],
          'clf__estimator__learning_rate': [1, 10],
          'clf__estimator__max_depth':[5, 10],
          #'clf__estimator__reg_alpha':[0.01, 1],
          #'clf__estimator__reg_lambda':[0.01, 1],
          #'clf__estimator__colsample_bytree':[0.85, 1],
         }


# grid Search
grid = GridSearchCV(pipe, param_grid=params,
                    cv=2, n_jobs=1, verbose=1,
                    return_train_score=True)
grid.fit(X_train, y_train)

Fitting 2 folds for each of 32 candidates, totalling 64 fits


[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed: 39.0min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...eg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)), 'features__text_pipeline__vect__max_features': (None, 5000), 'features__text_pipeline__tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [5], 'clf__estimator__learning_rate': [1, 10], 'clf__estimator__max_depth': [5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [145]:
grid.best_estimator_.named_steps['clf'].estimator

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=5,
       n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [140]:
y_pred = grid.predict(X_test)
classification_report_df = generate_classification_report(y_test, y_pred)
classification_report_df.T.style.background_gradient(cmap=cm, axis=1)

Mean f_score: 0.899748034898768
Mean precision: 0.8902927930015818
Mean recall: 0.9256539064818347


category,child_alone,offer,shops,tools,fire,hospitals,aid_centers,missing_people,clothing,cold,electricity,security,money,search_and_rescue,military,refugees,other_infrastructure,death,medical_products,transport,other_weather,buildings,water,infrastructure_related,earthquake,medical_help,floods,storm,shelter,food,other_aid,request,direct_report,weather_related,related,aid_related
f1_score,1,0.9939,0.992757,0.991931,0.985525,0.983054,0.98166,0.981154,0.97609,0.970464,0.969453,0.969392,0.964656,0.959674,0.950149,0.941865,0.93392,0.931902,0.928791,0.928724,0.92464,0.922417,0.910442,0.902531,0.892868,0.884368,0.877576,0.87151,0.866775,0.843823,0.812747,0.779222,0.747721,0.688918,0.668055,0.462254
precision,1,0.991877,0.990358,0.989345,0.980762,0.97749,0.975729,0.974977,0.968699,0.961203,0.959954,0.959468,0.953729,0.947275,0.937199,0.926988,0.913654,0.914544,0.906868,0.907087,0.904938,0.898649,0.89175,0.878214,0.901289,0.861527,0.85796,0.864502,0.837754,0.816942,0.780317,0.767752,0.737639,0.697294,0.677887,0.538919
recall,1,0.99593,0.995167,0.994531,0.990334,0.988681,0.987664,0.987409,0.983594,0.979906,0.979143,0.979524,0.975836,0.972402,0.965153,0.959812,0.955106,0.953071,0.9518,0.951418,0.94773,0.947476,0.939082,0.933104,0.915935,0.918225,0.913901,0.900801,0.908178,0.887702,0.865319,0.82246,0.800331,0.732926,0.753656,0.590233


### 9. Export your model as a pickle file

In [141]:
filename = 'models/pipeline_cv_model.pkl'
pickle.dump(grid, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.